# Ghost 7B Alpha

<p><img src="https://cdn-uploads.huggingface.co/production/uploads/600ae38cc92b79f54efd4556/QPrQZMQX_jzyYngmreP0_.jpeg" width="40%" align="center" /></p>

The large generation of language models focuses on optimizing excellent reasoning, multi-task knowledge, and tools support.

### Introduction

**Ghost 7B Alpha** is a large language model fine-tuned from Mistral 7B, with a size of 7 billion parameters. The model was developed with the goal of optimizing reasoning ability, multi-task knowledge and supporting tool usage. The model works well with the main trained and optimized languages being English and Vietnamese.

Overall, the model is suitable when making a pretrained version so you can continue to develop the desired tasks, develop virtual assistants, perform features on tasks such as coding, translation, answering questions, creating documents, etc. It is truly an efficient, fast and extremely cheap open model.

See details about the model [here](https://ghost-x.vercel.app/docs/models/ghost-7b-alpha/).

# Setup

First, to start experiencing the **Ghost 7B Alpha** model we need to install the necessary libraries, here will be transformers and related things.

## Transformers

Transformers provides thousands of pretrained models to perform tasks on different modalities such as text, vision, and audio. [[source]](https://github.com/huggingface/transformers)

These models can be applied on:

- 📝 Text, for tasks like text classification, information extraction, question answering, summarization, translation, and text generation, in over 100 languages.
- 🖼️ Images, for tasks like image classification, object detection, and segmentation.
- 🗣️ Audio, for tasks like speech recognition and audio classification.
Transformer models can also perform tasks on several modalities combined, such as table question answering, optical character recognition, information extraction from scanned documents, video classification, and visual question answering.

## Installation

Make the installation easy by running the following command.

In [ ]:
!pip install accelerate peft bitsandbytes transformers trl

## Downloads & More

Then, create and load the model from **HuggingFace** to use. In the code below, it will load and minify the text generation using functions created to condense the code for later use.

In [ ]:
import os, torch, pprint
from transformers import (
    pipeline,
    AutoTokenizer,
    AutoModelForCausalLM,
)

base_model = "ghost-x/ghost-7b-alpha"
tokenizer = AutoTokenizer.from_pretrained(base_model)
model = AutoModelForCausalLM.from_pretrained(
    base_model,
    torch_dtype=torch.bfloat16,
    low_cpu_mem_usage=True,
    device_map="auto",
)

pipe = pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
    torch_dtype=torch.bfloat16,
)

def generate_sample_response(
    messages,
    max_new_tokens=512,
    do_sample=True,
    temperature=0.2,
    top_k=50,
    top_p=0.95,
):
    prompt = pipe.tokenizer.apply_chat_template(
        messages, tokenize=False, add_generation_prompt=True
    )
    inputs = pipe.tokenizer(
        prompt,
        return_tensors="pt",
        add_special_tokens=False,
    )
    for k,v in inputs.items():
        inputs[k] = v.cuda()
    outputs = pipe.model.generate(
        **inputs,
        max_new_tokens=max_new_tokens,
        do_sample=do_sample,
        temperature=temperature,
        top_k=top_k,
        top_p=top_p,
    )
    results = pipe.tokenizer.batch_decode(outputs)[0]
    return results


def generate_default_response(
    messages,
    max_new_tokens=512,
):
    prompt = pipe.tokenizer.apply_chat_template(
        messages, tokenize=False, add_generation_prompt=True
    )
    inputs = pipe.tokenizer(
        prompt,
        return_tensors="pt",
        add_special_tokens=False,
    )
    for k,v in inputs.items():
        inputs[k] = v.cuda()
    outputs = pipe.model.generate(
        **inputs,
        max_new_tokens=max_new_tokens,
    )
    results = pipe.tokenizer.batch_decode(outputs)[0]
    return results

In [ ]:
import torch
torch.backends.cuda.enable_mem_efficient_sdp(False)
torch.backends.cuda.enable_flash_sdp(False)

# Playground

Now let's start to experience the model.

In the code below, we will generate text with the model without setting a system prompt for the model. These will represent generic use cases.

In [ ]:
question = """\
Theo bạn, xu hướng nào sẽ định hình thế giới trong 5 năm tới? Giải thích lý do cho lựa chọn của bạn.
"""

messages = [
    {"role": "system", "content": ""},
    {"role": "user", "content": question},
]

output = generate_sample_response(messages=messages, max_new_tokens=1024, temperature=0.4,
    top_k=50,
    top_p=0.95)
pprint.pprint(output)

Additionally, to try out the model with other role plays, start with the fun ones below.

### Try science and physics skills

Here, let's try to set up the system prompt by specifying the model as desired. In the example, we will set it as a **physics expert**. Of course, you can change and tweak to your liking to get different results, suitable to your wishes.

In [ ]:
question = """\
Hãy mô tả sự khác biệt giữa năng lượng tiềm năng và năng lượng động năng cho một đứa trẻ 12 tuổi. Nêu ví dụ về từng loại.
"""

messages = [
    # {"role": "system", "content": "You are Albert Einstein. You can easily explain complex things in the field of science and physics."},
    # {"role": "system", "content": "You are a scientist. You always answer questions accurately with explanations."},
    {"role": "system", "content": "You are a physics expert. You always use your knowledge to answer questions accurately and understandably."},
    {"role": "user", "content": question},
]

output = generate_sample_response(messages=messages, max_new_tokens=1024, temperature=0.4,
    top_k=50,
    top_p=0.95)
pprint.pprint(output)

In [ ]:
question = """\
Giải thích về dòng điện xoay chiều. Đưa ví dụ thật dễ hiểu.
"""

messages = [
    {"role": "system", "content": "You are Nikola Tesla."},
    {"role": "user", "content": question},
]

output = generate_sample_response(messages=messages, max_new_tokens=2048, temperature=0.4,
    top_k=50,
    top_p=0.95)
pprint.pprint(output)

### Try coding skills

Here, let's try to set up the system prompt by specifying the model as desired. In the example, we will set it as a **software engineer**. Of course, you can change and tweak to your liking to get different results, suitable to your wishes.

In [ ]:
question = """\
Viết mã để giải quyết bài toán sau: Cho một danh sách các số nguyên, hãy tìm vị trí của số lớn nhất và số nhỏ nhất trong danh sách, bằng Python.
"""

messages = [
    # {"role": "system", "content": "You are a software engineer."},
    # {"role": "system", "content": "You are a software engineer with 5 years of experience working at Google."},
    # {"role": "system", "content": "You are a computer science engineer. You have extensive knowledge of artificial intelligence and machine learning."},
    {"role": "system", "content": "You are a software engineer. You have extensive knowledge of computer science. You always answer questions accurately and professionally."},
    {"role": "user", "content": question},
]

output = generate_sample_response(messages=messages, max_new_tokens=1024, temperature=0.4,
    top_k=50,
    top_p=0.95)
pprint.pprint(output)

In [ ]:
question = """\
Giải thích cách sử dụng thư viện TensorFlow trong Python để xây dựng và huấn luyện một mạng nơ-ron nhân tạo cho bài toán phân loại văn bản.
"""

messages = [
    {"role": "system", "content": "You are a AI engineer. You have extensive knowledge of artificial intelligence and machine learning."},
    {"role": "user", "content": question},
]

output = generate_sample_response(messages=messages, max_new_tokens=2048, temperature=0.4,
    top_k=50,
    top_p=0.95)
pprint.pprint(output)

### Try reasoning skills

Here, let's try to set up the system prompt by specifying the model as desired. In the example, we will set it as a **science expert**. Of course, you can change and tweak to your liking to get different results, suitable to your wishes.

In [ ]:
question = """\
Một người đàn ông đang lái xe trên đường cao tốc thì xe của anh ta bị hỏng. Anh ta tấp vào lề đường và cố gắng sửa chữa xe, nhưng không thành công. Anh ta quyết định gọi cứu hộ, nhưng điện thoại của anh ta hết pin. Anh ta nhìn xung quanh và thấy có một ngôi nhà cách đó khoảng 500 mét. Anh ta đi bộ đến ngôi nhà và gõ cửa. Một người phụ nữ mở cửa và anh ta giải thích tình huống của mình. Người phụ nữ nói rằng cô ấy có thể giúp anh ta, nhưng cô ấy cần một số dụng cụ từ nhà kho của mình. Cô ấy bảo anh ta đợi ở cửa trong khi cô ấy đi lấy dụng cụ. Vài phút sau, người phụ nữ quay lại với một chiếc xô nước. Người đàn ông bối rối. Anh ta hỏi người phụ nữ, "Bà có thể giúp tôi bằng cách mang cho tôi một xô nước ư?" Người phụ nữ trả lời, "Chắc chắn rồi. Tôi sẽ đổ nước vào bình ắc quy của xe anh, và nó sẽ khởi động lại."

Câu hỏi: Giải thích tại sao phương pháp của người phụ nữ có thể giúp khởi động lại xe của người đàn ông ?
"""

messages = [
    # {"role": "system", "content": "You are the famous detective Conan. You help users solve difficult cases with your excellent reasoning and logic abilities."},
    # {"role": "system", "content": "You are a professional detective. You reason to find the cause of problems logically."},
    {"role": "system", "content": "You are a science expert. You are very good at reasoning to answer challenging questions."},
    {"role": "user", "content": question},
]

output = generate_sample_response(messages=messages, max_new_tokens=1024, temperature=0.4,
    top_k=50,
    top_p=0.95)
pprint.pprint(output)

In [ ]:
question = """\
Vụ án bí ẩn tại biệt thự cổ:

Nạn nhân là bà Lan Anh, một phụ nữ giàu có, được phát hiện đã chết trong thư phòng của mình tại biệt thự cổ. Cảnh sát đến hiện trường và tìm thấy một số manh mối:

Trên bàn làm việc của bà Lan Anh có một bức thư chưa gửi, được viết tay và có nội dung: "Tôi đã tìm ra bí mật của bạn. Hãy hối hận đi!".
Bên cạnh thi thể bà Lan Anh là một chiếc bình cổ quý giá, bị vỡ tan.
Dấu vân tay của người hầu tên Long được tìm thấy trên chiếc bình vỡ.
Long khai rằng anh ta đã dọn dẹp thư phòng vào sáng hôm xảy ra vụ án và vô tình va phải chiếc bình, khiến nó bị vỡ. Tuy nhiên, anh ta khẳng định không hề nhìn thấy bà Lan Anh và không biết chuyện gì đã xảy ra.
Không có dấu hiệu đột nhập hay vật lộn tại hiện trường.
Câu hỏi:

Ai là thủ phạm đã sát hại bà Lan Anh? Dựa trên những manh mối thu thập được, hãy suy luận logic và đưa ra lời giải thích hợp lý cho vụ án bí ẩn này.
"""

messages = [
    {"role": "system", "content": "You are Sherlock Holmes. You always answer logically with your brilliant reasoning."},
    {"role": "user", "content": question},
]

output = generate_sample_response(messages=messages, max_new_tokens=1024, temperature=0.4,
    top_k=50,
    top_p=0.95)
pprint.pprint(output)

### Try financial skills

Here, let's try to set up the system prompt by specifying the model as desired. In the example, we will set it as a **financial experts**. Of course, you can change and tweak to your liking to get different results, suitable to your wishes.

In [ ]:
question = """\
Theo bạn, những yếu tố nào ảnh hưởng đến giá trị đồng tiền Việt Nam so với các đồng tiền khác?
"""

messages = [
    # {"role": "system", "content": "You are the wolf of wall street. You always advise to help users make a lot of profit with low risk."},
    {"role": "system", "content": "You are a financial expert. You always give advice carefully, after carefully examining the facts."},
    {"role": "user", "content": question},
]

output = generate_sample_response(messages=messages, max_new_tokens=2048, temperature=0.4,
    top_k=50,
    top_p=0.95)
pprint.pprint(output)

In [ ]:
question = """\
Dưới đây là thông tin các loại thẻ ngân hàng VOB đang cung cấp.

| THẺ                        | VIB Super Card                                                                                                                          | VIB Cash Back                                                                                                                | VIB Premier Boundless                                                               |
| -------------------------- | --------------------------------------------------------------------------------------------------------------------------------------- | ---------------------------------------------------------------------------------------------------------------------------- | ----------------------------------------------------------------------------------- |
| HẠN MỨC GIAO DỊCH          | Hạn mức lên đến 600 triệu đồng                                                                                                          | Hạn mức lên đến 600 triệu đồng                                                                                               | Hạn mức lên đến 2 tỷ đồng                                                           |
| TÍNH NĂNG NỔI BẬT          |                                                                                                                                         |                                                                                                                              |                                                                                     |
|                            | Hoàn linh hoạt đến 15% cho chi tiêu Mua sắm, Du lịch, Ăn uống                                                                           | Tùy chọn Tỉ lệ thanh toán tối thiểu và Ngày sao kê                                                                           | Miễn phí chọn Số Thẻ đẹp                                                            |
|                            | Hoàn đến 24 triệu điểm thưởng/năm                                                                                                       | Tích lũy dặm Vietnam Airlines cho mọi chi tiêu                                                                               | Tặng 4.000 dặm Bông Sen Vàng                                                        |
|                            |                                                                                                                                         | Miễn phí nâng hạng thẻ Titan, Gold, Platinum chương trình Bông Sen Vàng của Vietnam Airlines                                 |                                                                                     |
| PHÍ THƯỜNG NIÊN            | 999.000 VNĐ                                                                                                                             | 899.000 VNĐ                                                                                                                  | 1.299.000 VNĐ                                                                       |
| LÃI SUẤT                   | 03 Kỳ sao kê đầu: 2.96%/tháng                                                                                                           | 2.83%/tháng Miễn lãi lên đến 55 ngày                                                                                         | 2.83%/ tháng Miễn lãi lên đến 55 ngày                                               |
| CHƯƠNG TRÌNH HOÀN TIỀN,... | Hoàn đến 15% chi tiêu tại Mua sắm, Du lịch, Ăn uống                                                                                     | Hoàn tiền không giới hạn                                                                                                     | Hoàn tiền đến 10% cho nhóm danh mục chi tiêu đăng ký                                |
|                            |                                                                                                                                         | Hoàn tiền 0,1% cho toàn bộ số tiền chi tiêu hợp lệ                                                                           |                                                                                     |
|                            | Tích lũy dặm thưởng cho mọi chi tiêu                                                                                                    |                                                                                                                              |                                                                                     |
|                            | 25.000 VNĐ chi tiêu tại Vietnam Airlines = 2 dặm thưởng                                                                                 |                                                                                                                              |                                                                                     |
|                            | 25.000 VNĐ chi tiêu tại nước ngoài = 1,5 dặm thưởng                                                                                     |                                                                                                                              |                                                                                     |
|                            | 25.000 VNĐ chi tiêu trong nước = 1 dặm thưởng                                                                                           |                                                                                                                              |                                                                                     |
|                            | Tích lũy dặm thưởng không giới hạn                                                                                                      |                                                                                                                              |                                                                                     |
| ĐỔI ĐIỂM THƯỞNG            | Bạn có thể chủ động đổi số tiền hoàn tích lũy thành tiền mặt, e-voucher, phí thường niên thông qua MyVIB với thao tác chưa đầy 30 giây. | Bạn có thể chủ động đổi số tiền hoàn tích lũy thành tiền mặt, phí thường niên thông qua MyVIB với thao tác chưa đầy 30 giây. | Dặm thưởng được tự động cộng vào tài khoản Hội viên Bông Sen Vàng sau mỗi kỳ sao kê |

Câu hỏi: Tôi đang là quản lý dự án, công việc thường xuyên đi lại, mức thu nhập khoảng 40/tháng. Hãy tư vấn cho tôi loại thẻ phù hợp.
"""

messages = [
    {"role": "system", "content": "You are a banking consultant. You are always honest and give the most beneficial and appropriate advice to users."},
    {"role": "user", "content": question},
]

output = generate_sample_response(messages=messages, max_new_tokens=1024, temperature=0.4,
    top_k=50,
    top_p=0.95)
pprint.pprint(output)

### Try literature and art skills

Here, let's try to set up the system prompt by specifying the model as desired. In the example, we will set it as an **expert in literature and art**. Of course, you can change and tweak to your liking to get different results, suitable to your wishes.

In [ ]:
question = """\
Trăm năm trong cõi người ta
Chữ tài chữ mệnh khéo là ghét nhau.

Câu hỏi: Tác giả là ai ? Giới thiệu về tác phẩm.
"""

messages = [
    # {"role": "system", "content": "You are a famous artist. You have extensive knowledge of works of art."},
    # {"role": "system", "content": "You are a famous literary researcher. You can answer your knowledge of literature accurately."},
    {"role": "system", "content": "You are an expert in literature and art."},
    {"role": "user", "content": question},
]

output = generate_sample_response(messages=messages, max_new_tokens=1024, temperature=0.2,
    top_k=50,
    top_p=0.95)
pprint.pprint(output)

In [ ]:
question = """\
Bạn ấn tượng về tác phẩm nào của mình nhất. Giải thích tại sao ?
Nếu được thay đổi đổi một phần nội dung, bạn sẽ thay đổi nó thế nào ?
"""

messages = [
    {"role": "system", "content": "You are William Shakespeare."},
    {"role": "user", "content": question},
]

output = generate_sample_response(messages=messages, max_new_tokens=1024, temperature=0.4,
    top_k=50,
    top_p=0.95)
pprint.pprint(output)

### Try medical and health skills

Here, let's try to set up the system prompt by specifying the model as desired. In the example, we will set it as an **medical experts**. Of course, you can change and tweak to your liking to get different results, suitable to your wishes.

In [ ]:
question = """\
Bệnh nhân: Nam, 35 tuổi, đến khám vì bị sốt cao 39°C liên tục trong 3 ngày, kèm theo ho khan, đau đầu, đau nhức cơ thể và mệt mỏi. Bệnh nhân không có tiền sử mắc bệnh lý nền, không dị ứng với thuốc.

Khám lâm sàng:
Nhiệt độ: 39°C
Huyết áp: 120/80 mmHg
Mạch: 90 lần/phút
Nhịp thở: 20 lần/phút
Khám phổi: nghe rào ron ran ở cả hai phổi
Tim: nhịp tim đều, không tiếng tim bất thường
Bụng: mềm, không đau
Da: không phát ban
Câu hỏi:

Dựa trên thông tin trên, bạn nghĩ bệnh nhân có thể mắc bệnh gì? Hãy giải thích lý do cho lựa chọn của bạn và đề xuất các xét nghiệm cần thiết để chẩn đoán chính xác.
"""

messages = [
    # {"role": "system", "content": "You are a health expert. You like to help and always give accurate health and nutrition information."},
    {"role": "system", "content": "You are a medical professional. You always give scientific and useful advice."},
    {"role": "user", "content": question},
]

output = generate_sample_response(messages=messages, max_new_tokens=1024, temperature=0.2,
    top_k=50,
    top_p=0.95)
pprint.pprint(output)

In [ ]:
question = """\
Hãy tóm gọn thành tựu cả đời của bạn để tôi biết. Nếu có thể sáng chế ra một loại thuốc, bạn sẽ sáng chế ra loại thuốc gì ?
"""

messages = [
    {"role": "system", "content": "You are Alexandre Yersin."},
    {"role": "user", "content": question},
]

output = generate_sample_response(messages=messages, max_new_tokens=2048, temperature=0.2,
    top_k=50,
    top_p=0.95)
pprint.pprint(output)